### OBTENCION DE DATOS POKEAPI

#### Importacion de librerias necesarias

In [1]:
import pandas as pd
import requests
import json

#### Conociendo la API

El objetivo de esta seccion es obtener el listado completo de pokemons desde la API publica PokeApi (*https://pokeapi.co/*). Para este fin primero he leido la documentacion de la API para entender su funcionamiento y la forma en que se realizan los request ya que el resultado de la peticion varia dependediendo el paramentro enviado.<br>Una vez que definí los datos de interes de cada pokemon, he logrado determinar que para conseguir la informacion que deseo basta con dos request por pokemon. Entendiendo esto se define la funcion **get_data()** que realiza dos request por cada ID de pokemon y devuelve un diccionario con la respuesta de ambas solicitudes. Cada pokemon cuenta con un identificador y estos id están comprendidos en el rango del 1 al 1025.

In [2]:
def get_pokemon_data(id:int) -> dict:
    url = f'https://pokeapi.co/api/v2/pokemon/{id}/'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f'Codigo de error: {response.status_code} en id: {id}')
    
    data = {'Basic': response.json()}
    url = f'https://pokeapi.co/api/v2/pokemon-species/{id}/'
    response = requests.get(url)    
    if response.status_code != 200:
        raise Exception(f'Codigo de error: {response.status_code} en id: {id}')    
    data['Plus'] = response.json()    
    return data

#### Obtencion de la informacion

En esta seccion obtendremos la informacion general de cada uno de los pokemons para ello se crea la lista **data_pkm** que se encarga de alojar los diccionarios resultantes de la funcion **get_data()** que posteriormente se ejecuta en un ciclo for con una cantidad de 1025 iteraciones.

In [3]:
#LISTA PARA GUARDAR LA INFORMACION OBTENIDA DE LA REQUEST
pkm_data = []

#CICLO PARA OBTENER INFORMACIONDE LOS 1025 POKEMONS
for id in range(1,1026):
    try:    
        pkm_data.append(get_pokemon_data(id))
    except Exception as ex:
        print(ex)

#CANTIDAD DE ELEMENTOS QUE SE OBTUVIERON
print(f"La cantidad de elementos en la lista es: {len(pkm_data)}")

La cantidad de elementos en la lista es: 1025


Se guarda la informacion obtenida en un archivo tipo **JSON** de manera local

In [4]:
with open('.\Json\PokeApiData.json','w') as file:
    json.dump(pkm_data, file, indent=4, separators=(',',':'))

#### Extraccion de los datos de interes

Con el objetivo de extraer y estructurar los datos de interes que se obtubieron anteriormente se crea la clase **Pokemon()** quien se encarga de alojar mediante sus atributos los datos de interes. Tambien se crea la clase **PokemonBuilder()** la cual se encarga de extrer los datos de cada pokemon desde el diccionario que recibe por parametro, para luego devolver una instancia de la clase **Pokemon()** con sus atributos ya seteados.

In [5]:
class Pokemon():
    def __init__(self):
        self.id = None
        self.name = None
        self.evolves_from = None        
        self.generation = None        
        self.type1 = None
        self.type2 = None        
        self.hp = None
        self.atk = None
        self.Def = None
        self.spatk = None
        self.spdef = None
        self.speed = None
        self.total = None
        self.is_legenday = None
        self.is_mythical = None        
        self.abilities = None
        self.egg_groups = None
        self.varietes = None

class PokemonBuilder():
    def __init__(self, pkm_data : dict):
        self.pkm_data = pkm_data

    def build(self) -> Pokemon:
        pkm = Pokemon()
        pkm.id = self.__get_id()
        pkm.name = self.__get_name()
        pkm.is_legenday = self.__is_legendary()
        pkm.is_mythical = self.__is_mythical()
        pkm.generation = self.__get_generation()
        pkm.evolves_from = self.__get_evolve_from()
        pkm.type1 = self.__get_type1()
        pkm.type2 = self.__get_type2()
        pkm.egg_groups = self.__get_egg_group()
        pkm.hp = self.__get_hp_stat()
        pkm.atk = self.__get_atk_stat()
        pkm.Def = self.__get_def_stat()
        pkm.spatk = self.__get_spatk_stat()
        pkm.spdef = self.__get_spdef_stat()
        pkm.speed = self.__get_spd_stat()
        pkm.total = self.__get_total_stat(pkm.hp, pkm.atk, pkm.Def, pkm.spatk, pkm.spdef, pkm.speed)
        pkm.abilities = self.__get_abilities()
        pkm.varietes = self.__get_varietes()
        return pkm

    def __get_id(self):
        return self.pkm_data.get('Basic', None).get('id', None)
    
    def __get_name(self):
        return self.pkm_data.get('Basic', None).get('name', None)
    
    def __get_type1(self): 
        return self.pkm_data['Basic']['types'][0]['type']['name']
    
    def __get_type2(self):
        if len(self.pkm_data['Basic']['types']) > 1:
            return self.pkm_data['Basic']['types'][1]['type']['name']
        else:
            return None  

    def __get_hp_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'hp':
                return item['base_stat']
        return None   
    
    def __get_atk_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'attack':
                return item['base_stat']
        return None
    
    def __get_def_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'defense':
                return item['base_stat']
        return None
    
    def __get_spatk_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'special-attack':
                return item['base_stat']
        return None
    
    def __get_spdef_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'special-defense':
                return item['base_stat']
        return None
    
    def __get_spd_stat(self): 
        for item in self.pkm_data['Basic']['stats']:
            if item['stat']['name'] == 'speed':
                return item['base_stat']
        return None
    
    def __get_total_stat(self, hp, atk, defe, spatk, spdef,speed):
        return hp + atk + defe + spatk + spdef + speed

    def __get_abilities(self):
        abilities = None
        for item in self.pkm_data['Basic']['abilities']:
            if abilities is None:
                abilities = item['ability']['name']
            else:
                abilities = abilities + '_' + item['ability']['name']
        return abilities
    
    def __is_legendary(self):
        return self.pkm_data['Plus']['is_legendary']
    
    def __is_mythical(self):
        return self.pkm_data['Plus']['is_mythical']
    
    def __get_generation(self):
        return self.pkm_data['Plus'].get('generation', {}).get('name', None)
    
    def __get_evolve_from(self):
        if self.pkm_data.get('Plus',{}).get('evolves_from_species', None) is not None:
            return self.pkm_data.get('Plus',{}).get('evolves_from_species', {}).get('name', None)
        else:
            return None
    
    def __get_egg_group(self):
        egg_groups = None
        for item in self.pkm_data['Plus']['egg_groups']:
            if egg_groups is None:            
                egg_groups = item['name']
            else:
                egg_groups = egg_groups + '_' + item['name']
        return egg_groups

    def __get_varietes(self):        
        if len(self.pkm_data['Plus']['varieties']) <= 1:
            return None
        varietes = None
        for item in self.pkm_data['Plus']['varieties']:
            if item['is_default'] == True:
                continue 
            if varietes is None:
                varietes = item['pokemon']['name']
            else:
                varietes = varietes + '_'+ item['pokemon']['name']
        return varietes      

Se crea la lista **pokemons** con el objetivo de almacenar las instancias devueltas por la clase **PokemonBuilder()** cpn forma de diccionario. Se crea un ciclo for a fines de recorrer la lista **data_pkm** y por cada elemento en esta lista obtener una instancia de la clase **Pokemon()**

In [6]:
#SE CREA UNA LISTA DE CLASES POKEMON CON LA INFORMACION DE INTERES
pokemons = [PokemonBuilder(pkm).build() for pkm in pkm_data]

#TRANSFORMACION DE CLASES A DICCIONARIOS PARA POSTERIOR GUARDADO EN JSON
pokemons = [vars(pkm) for pkm in pokemons]

Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [7]:
with open('PokemonData.json','w') as file:
    json.dump(pokemons, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**pokemons**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [8]:
df = pd.DataFrame(pokemons)
df.head()

,id,name,evolves_from,generation,type1,type2,hp,atk,Def,spatk,spdef,speed,total,is_legenday,is_mythical,abilities,egg_groups,varietes
0,1,bulbasaur,None,generation-i,grass,poison,45,49,49,65,65,45,318,False,False,overgrow_chlorophyll,monster_plant,None
1,2,ivysaur,bulbasaur,generation-i,grass,poison,60,62,63,80,80,60,405,False,False,overgrow_chlorophyll,monster_plant,None
2,3,venusaur,ivysaur,generation-i,grass,poison,80,82,83,100,100,80,525,False,False,overgrow_chlorophyll,monster_plant,venusaur-mega_venusaur-gmax
3,4,charmander,None,generation-i,fire,None,39,52,43,60,50,65,309,False,False,blaze_solar-power,monster_dragon,None
4,5,charmeleon,charmander,generation-i,fire,None,58,64,58,80,65,80,405,False,False,blaze_solar-power,monster_dragon,None


Tambien se puede leer el archivo **JSON** creado anteriormente utilizando un **DataFrame** de **Pandas**

In [9]:
df = pd.read_json('PokemonData.json')
df.head()

,id,name,evolves_from,generation,type1,type2,hp,atk,Def,spatk,spdef,speed,total,is_legenday,is_mythical,abilities,egg_groups,varietes
0,1,bulbasaur,None,generation-i,grass,poison,45,49,49,65,65,45,318,False,False,overgrow_chlorophyll,monster_plant,None
1,2,ivysaur,bulbasaur,generation-i,grass,poison,60,62,63,80,80,60,405,False,False,overgrow_chlorophyll,monster_plant,None
2,3,venusaur,ivysaur,generation-i,grass,poison,80,82,83,100,100,80,525,False,False,overgrow_chlorophyll,monster_plant,venusaur-mega_venusaur-gmax
3,4,charmander,None,generation-i,fire,None,39,52,43,60,50,65,309,False,False,blaze_solar-power,monster_dragon,None
4,5,charmeleon,charmander,generation-i,fire,None,58,64,58,80,65,80,405,False,False,blaze_solar-power,monster_dragon,None
